### Importar librerías

In [10]:
# =====================================
# IMPORTAR LIBRERÍAS
# =====================================
import pandas as pd
import os
# =====================================
# CARGA DE DATOS
# =====================================
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/ACE.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/ACE.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/ACE.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/ACE.xlsx


### Limpieza de datos

In [11]:
# =====================================
# LIMPIEZA Y FORMATEO DE DATOS
# =====================================
df.columns = df.columns.str.replace(r"\n", " ", regex=True).str.strip()

cols_drop = [
    "Unnamed: 0",
    "Temp. De muestra (°C)",
    "Examen Visual (ASTM 1524)",
    "Gravedad Específica (ASTM D1298, g/mL)",
    "Humedad ambiente (%)",
]
df = df.drop(columns=cols_drop)

if 'FECHA DE MUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE MUESTRA': 'FECHA'})
elif 'FECHA DE\nMUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE\nMUESTRA': 'FECHA'})
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
df["TENSION"] = df["TENSION"].str.split("/").str[0]

rename_map = {
    "Contenido de humedad (ASTM D1533, ppm)": "HU",
    "Rigidez Dieléctrica (ASTM D1816, kV/2 mm)": "RD",
    "Tensión Interfacial (ASTM D971, mN/m)": "TIF",
    "Indice de neutralización (ASTM D974, mg KOH/g)": "AC",
    "Color (ASTM D1500)": "CO",
    "Factor de potencia a 25°C (ASTM D924, %)": "FP25",
    "Factor de potencia a 100°C (ASTM D924, %)": "FP100",
    "Contenido de inhibidor (ASTM D-2668)": "IO",
}
df = df.rename(columns=rename_map)

orden = ["SERIE", "TENSION", "FECHA", "FP25", "FP100", "HU", "AC", "TIF", "CO", "RD", "IO"]
df = df[orden]

num_cols = ["FP25", "FP100", "HU", "AC", "TIF", "CO", "RD", "IO"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
df["SERIE"] = df["SERIE"].astype(str)
# Guardar tabla original sin tensión
# df_full = df.drop(columns=["TENSION"]).copy()
df_full = df.drop(columns=["TENSION"]).copy()
df_full.head()

,SERIE,FECHA,FP25,FP100,HU,AC,TIF,CO,RD,IO
0,D518293,2025-02-07,0.002,0.096,6.0,0.00,39.9,0.5,74.0,NaN
1,D518293,2023-08-29,0.001,0.034,7.0,0.01,39.6,0.5,74.0,NaN
2,D518293,2022-04-28,0.004,0.080,7.0,0.02,41.3,1.0,71.2,NaN
3,D518293,2021-08-15,0.007,0.225,14.0,0.02,40.0,0.5,42.0,NaN
4,D518293,2019-04-25,0.005,0.060,NaN,0.01,45.9,0.5,64.0,NaN


### Lógica

In [12]:
# =====================================
# PUNTAJES IEEE
# =====================================
score = [5, 3, 1]

reglas = {
    "220": {
        "FP25": [(lambda x: x > 0.3, score[0]), (lambda x: 0.1 < x <= 0.3, score[1]), (lambda x: x <= 0.1, score[2])],
        "FP100": [(lambda x: x > 4, score[0]), (lambda x: 3 < x <= 4, score[1]), (lambda x: x <= 3, score[2])],
        "HU": [(lambda x: x > 30, score[0]), (lambda x: 20 < x <= 30, score[1]), (lambda x: x <= 20, score[2])],
        "AC": [(lambda x: x > 0.10, score[0]), (lambda x: 0.05 < x <= 0.10, score[1]), (lambda x: x <= 0.05, score[2])],
        "TIF": [(lambda x: x < 28, score[0]), (lambda x: 28 <= x < 32, score[1]), (lambda x: x >= 32, score[2])],
        "CO": [(lambda x: x > 3.5, score[0]), (lambda x: x <= 3.5, score[2])],
        "RD": [(lambda x: x < 45, score[0]), (lambda x: 45 <= x < 50, score[1]), (lambda x: x >= 50, score[2])],
        "IO": [(lambda x: x < 0.1, score[0]), (lambda x: 0.1 <= x < 0.2, score[1]), (lambda x: x >= 0.2, score[2])],
    },
    "60": {
        "FP25": [(lambda x: x > 0.3, score[0]), (lambda x: 0.1 < x <= 0.3, score[1]), (lambda x: x <= 0.1, score[2])],
        "FP100": [(lambda x: x > 4, score[0]), (lambda x: 3 < x <= 4, score[1]), (lambda x: x <= 3, score[2])],
        "HU": [(lambda x: x > 40, score[0]), (lambda x: 30 < x <= 40, score[1]), (lambda x: x <= 30, score[2])],
        "AC": [(lambda x: x > 0.10, score[0]), (lambda x: 0.05 < x <= 0.10, score[1]), (lambda x: x <= 0.05, score[2])],
        "TIF": [(lambda x: x < 28, score[0]), (lambda x: 28 <= x < 32, score[1]), (lambda x: x >= 32, score[2])],
        "CO": [(lambda x: x > 3.5, score[0]), (lambda x: x <= 3.5, score[2])],
        "RD": [(lambda x: x < 35, score[0]), (lambda x: 35 <= x < 40, score[1]), (lambda x: x >= 40, score[2])],
        "IO": [(lambda x: x < 0.1, score[0]), (lambda x: 0.1 <= x < 0.2, score[1]), (lambda x: x >= 0.2, score[2])],
    },
}

def puntaje_parametro(valor, tension, parametro):
    if pd.isna(valor):
        return float("nan")
    reglas_tension = reglas.get(str(tension), {})
    condiciones = reglas_tension.get(parametro, [])
    for condicion, puntaje in condiciones:
        if condicion(valor):
            return puntaje
    return 1

for p in num_cols:
    df[f"Puntaje_{p}"] = df.apply(lambda row: puntaje_parametro(row[p], row["TENSION"], p), axis=1)
df_otro = df.copy()
df_otro.head()

,SERIE,TENSION,FECHA,FP25,FP100,HU,AC,TIF,CO,RD,IO,Puntaje_FP25,Puntaje_FP100,Puntaje_HU,Puntaje_AC,Puntaje_TIF,Puntaje_CO,Puntaje_RD,Puntaje_IO
0,D518293,220,2025-02-07,0.002,0.096,6.0,0.00,39.9,0.5,74.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1,D518293,220,2023-08-29,0.001,0.034,7.0,0.01,39.6,0.5,74.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,D518293,220,2022-04-28,0.004,0.080,7.0,0.02,41.3,1.0,71.2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
3,D518293,220,2021-08-15,0.007,0.225,14.0,0.02,40.0,0.5,42.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,5.0,NaN
4,D518293,220,2019-04-25,0.005,0.060,NaN,0.01,45.9,0.5,64.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN


### Asignación de puntaje

In [13]:
def puntaje_parametro(valor, tension, parametro):
    if pd.isna(valor):
        return float("nan")
    reglas_tension = reglas.get(str(tension), {})
    condiciones = reglas_tension.get(parametro, [])
    for condicion, puntaje in condiciones:
        if condicion(valor):
            return puntaje
    return 1  # valor por defecto

# Crear puntajes
parametros = num_cols
for p in parametros:
    df[f"Puntaje_{p}"] = df.apply(lambda row: puntaje_parametro(row[p], row["TENSION"], p), axis=1)
# =====================================
# CÁLCULO ACE
# =====================================
pesos = {
    "Puntaje_FP25": 3,
    "Puntaje_FP100": 3,
    "Puntaje_HU": 4,
    "Puntaje_AC": 2,
    "Puntaje_TIF": 3,
    "Puntaje_CO": 1,
    "Puntaje_RD": 5,
    "Puntaje_IO": 1,
}
df.head()

,SERIE,TENSION,FECHA,FP25,FP100,HU,AC,TIF,CO,RD,IO,Puntaje_FP25,Puntaje_FP100,Puntaje_HU,Puntaje_AC,Puntaje_TIF,Puntaje_CO,Puntaje_RD,Puntaje_IO
0,D518293,220,2025-02-07,0.002,0.096,6.0,0.00,39.9,0.5,74.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1,D518293,220,2023-08-29,0.001,0.034,7.0,0.01,39.6,0.5,74.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,D518293,220,2022-04-28,0.004,0.080,7.0,0.02,41.3,1.0,71.2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
3,D518293,220,2021-08-15,0.007,0.225,14.0,0.02,40.0,0.5,42.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,5.0,NaN
4,D518293,220,2019-04-25,0.005,0.060,NaN,0.01,45.9,0.5,64.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN


### Cálculo ACE

In [14]:
def calcular_ACE(row, pesos):
    valores = [row[col] * peso for col, peso in pesos.items() if pd.notna(row[col])]
    total_peso = sum(peso for col, peso in pesos.items() if pd.notna(row[col]))
    return sum(valores) / total_peso if total_peso > 0 else float("nan")

df["ACE"] = df.apply(lambda row: calcular_ACE(row, pesos), axis=1)
df_ACE = df[["SERIE", "FECHA", "ACE"]]
df_ACE.head()

,SERIE,FECHA,ACE
0,D518293,2025-02-07,1.000000
1,D518293,2023-08-29,1.000000
2,D518293,2022-04-28,1.000000
3,D518293,2021-08-15,1.952381
4,D518293,2019-04-25,1.000000


### Extensión de Calendario

### Transformación de datos y extensión del calendario desde 2015 al dia de hoy, esta fecha(2015) se puede cambiar, al hacerlo, se debe cambiar en todas las tablas(ACE,AIS,ARR,ARRdis,NUCiex,etc...)

In [ ]:
# =====================================
# EXTENSIÓN DEL CALENDARIO (DESDE 2025)
# =====================================
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE","FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# Última medición antes de 2025
ultimos_2024 = df_ACE[df_ACE['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_ACE, ultimos_2024], ignore_index=True)
df_extendida = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# Extender detalles
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)
df_extendida_detalles = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
df_extendida_detalles.tail()

# =====================================
# DETALLES + ACE
# =====================================
df_detalles = pd.merge(df_full, df_ACE, on=["SERIE","FECHA"], how="left")
df_detalles_ext = pd.merge(df_extendida_detalles, df_extendida, on=["SERIE","FECHA"], how="left")

# Reordenar para que ACE quede después de FECHA
def reordenar(df_in):
    cols = list(df_in.columns)
    if "ACE" in cols:
        cols.remove("ACE")
        idx = cols.index("FECHA") + 1
        cols = cols[:idx] + ["ACE"] + cols[idx:]
    return df_in[cols]

df_detalles = reordenar(df_detalles)
df_detalles_ext = reordenar(df_detalles_ext)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_15032\3182850626.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_15032\3182850626.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill())

,SERIE,FECHA,FP25,FP100,HU,AC,TIF,CO,RD,IO
641525,V180403,2025-10-30,0.001,0.069,3.0,0.003,42.8,0.5,75.0,NaN
641526,V180403,2025-10-31,0.001,0.069,3.0,0.003,42.8,0.5,75.0,NaN
641527,V180403,2025-11-01,0.001,0.069,3.0,0.003,42.8,0.5,75.0,NaN
641528,V180403,2025-11-02,0.001,0.069,3.0,0.003,42.8,0.5,75.0,NaN
641529,V180403,2025-11-03,0.001,0.069,3.0,0.003,42.8,0.5,75.0,NaN


### Funciones a llamar

In [17]:
# =====================================
# FUNCIONES
# =====================================
def get_df_ACE():
    return df_ACE

def get_df_extendida_ACE():
    return df_extendida

def get_df_detalles_ACE():
    return df_detalles

def get_df_detalles_ext_ACE():
    return df_detalles_ext

In [18]:
# =====================================
# PRUEBA
# =====================================
print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_ACE())
print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_ACE().head())
print('\n ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_ACE().head())
print('\n ====== TABLA DE DETALLES CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_ACE().head())



 ====== TABLA CON FECHAS ORIGINALES ====== 

          SERIE      FECHA       ACE
0       D518293 2025-02-07  1.000000
1       D518293 2023-08-29  1.000000
2       D518293 2022-04-28  1.000000
3       D518293 2021-08-15  1.952381
4       D518293 2019-04-25  1.000000
...         ...        ...       ...
2932  230531-01 2005-06-08  5.000000
2933  230531-01 2005-05-07  5.000000
2934  230531-01 2004-06-08  1.000000
2935  230531-01 2004-06-03  1.000000
2936  230531-01 2004-06-01  1.000000

[2937 rows x 3 columns]

 ====== TABLA CON FECHAS EXTENDIDAS ====== 

    SERIE      FECHA  ACE
0  100138 2015-01-01  NaN
1  100138 2015-01-02  NaN
2  100138 2015-01-03  NaN
3  100138 2015-01-04  NaN
4  100138 2015-01-05  NaN

 ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== 

     SERIE      FECHA       ACE   FP25  FP100    HU    AC   TIF   CO    RD  IO
0  D518293 2025-02-07  1.000000  0.002  0.096   6.0  0.00  39.9  0.5  74.0 NaN
1  D518293 2023-08-29  1.000000  0.001  0.034   7.0  0.01  39.6  0.